In [66]:
#!python -m spacy download ru_core_news_sm 

In [67]:
import pandas as pd
import pickle
import os.path 
import re
import spacy
import Levenshtein
import tqdm

nlp = spacy.load("ru_core_news_sm")

In [68]:
def read_pickle(texts: str):
    # Открытие файла texts.pickle для чтения в бинарном режиме
    with open(os.path.join(data_path,texts), 'rb') as file:
        # Загрузка данных из файла и возвращение их
        return pickle.load(file)

data_path = "hakaton-gagarin-sentiment_interface\data"

# Преобразование загруженных данных в DataFrame
dfmentions = pd.DataFrame(read_pickle('mentions_texts.pickle'))
dfsetiments = pd.DataFrame(read_pickle('sentiment_texts.pickle'))

In [69]:
dfsetiments

MessageID   ChannelID  issuerid  SentimentScore           DateAdded  \
0        241407  1203560567       153               2 2023-05-12 19:03:20   
1         33684  1136626166       230               4 2023-02-03 20:56:29   
2         10090  1063908560       118               4 2023-06-02 19:18:37   
3         10090  1063908560       220               5 2023-06-02 19:18:37   
4          9826  1063908560        89               2 2023-04-24 17:51:38   
...         ...         ...       ...             ...                 ...   
9284      47482  1197210433       157               4 2023-03-20 14:53:14   
9285     233829  1203560567       157               4 2023-03-20 14:58:04   
9286       9789  1063908560       225               3 2023-04-19 17:51:56   
9287     233867  1203560567       127               3 2023-03-20 14:58:04   
9288      14315  1075101206       112               5 2023-02-07 14:56:26   

              DatePosted                                        MessageText  \
0    2023-05-12 19:02:42  ⚠️🇷🇺#SELG #дивиденд  сд Селигдар: дивиденды 20...   
1    2023-02-03 16:46:34  Ozon продолжает развивать специализированные ф...   
2    2023-06-02 18:50:00  ​Фокусы продолжаются🔥Акции и инвестиции  📈ВТБ ...   
3    2023-06-02 18:50:00  ​Фокусы продолжаются🔥Акции и инвестиции  📈ВТБ ...   
4    2023-04-24 13:54:00  ​​Windfall Tax — налог на сверхприбыль. Какие ...   
...                  ...                                                ...   
9284 2023-03-20 12:15:21  #FLOT #Дивиденды 💰 7% — возможная дивдоходност...   
9285 2023-03-20 12:05:49  🇷🇺#FLOT #отчетность  ЧИСТАЯ ПРИБЫЛЬ СОВКОМФЛОТ...   
9286 2023-04-19 15:32:00  ​​Ключевой принцип создания портфеля  🔹Диверси...   
9287 2023-03-20 14:33:32  "💥🇷🇺#PLZL #листинг #торги  ""Полюс"" ведет диа...   
9288 2023-02-07 14:28:09  Роснефть (ROSN) справедливая цена 425.17 рубля...   

      IsForward  
0         False  
1         False  
2         False  
3         False  
4         False  
...         ...  
9284      False  
9285      False  
9286      False  
9287      False  
9288      False  

[9289 rows x 8 columns]

In [70]:
name_company = pd.read_excel('hakaton-gagarin-sentiment_interface\data\\names and synonyms.xlsx')

In [71]:
dict_company_names = {row[0]: [val for val in row[1:] if pd.notna(val)] for row in name_company.values.tolist()}
dict_company_names

{1: ['Акционерный коммерческий банк "Держава" публичное акционерное общество',
  'DERZP',
  'Держава',
  'DERZHAVA',
  'DERZ'],
 2: ['"МОСКОВСКИЙ КРЕДИТНЫЙ БАНК" (публичное акционерное общество)',
  'CBOM',
  'CBOM RX',
  'Московский кредитный банк',
  'мкб',
  'Credit Bank of Moscow',
  'Credit Bank'],
 3: ['"Российский акционерный коммерческий дорожный банк" (публичное акционерное общество)',
  'RDRB',
  'Российский акционерный коммерческий дорожный банк',
  'РДБанк',
  'Дорожный банк',
  'Russian public joint-stock commercial roads Bank',
  'RosDorBank',
  'РосДорБанк',
  'roads Bank'],
 4: ['Акционерная компания "АЛРОСА" (публичное акционерное общество)',
  'ALRS',
  'ALRS RX',
  'алроса',
  'alrosa'],
 5: ['Акционерный Коммерческий банк "АВАНГАРД" - публичное акционерное общество',
  'AVAN',
  'Авангард',
  'AVANGARD'],
 6: ['акционерный коммерческий банк "Приморье"  (публичное акционерное общество)',
  'PRMB',
  'АКБ Приморье',
  'АКБ "Приморье"',
  'банк "Приморье"',
  'банк При

In [72]:
company_names = sum([[val for i, val in enumerate(row) if pd.notna(val) and i!=0] for row in name_company.values.tolist()], [])
company_names

['Акционерный коммерческий банк "Держава" публичное акционерное общество',
 'DERZP',
 'Держава',
 'DERZHAVA',
 'DERZ',
 '"МОСКОВСКИЙ КРЕДИТНЫЙ БАНК" (публичное акционерное общество)',
 'CBOM',
 'CBOM RX',
 'Московский кредитный банк',
 'мкб',
 'Credit Bank of Moscow',
 'Credit Bank',
 '"Российский акционерный коммерческий дорожный банк" (публичное акционерное общество)',
 'RDRB',
 'Российский акционерный коммерческий дорожный банк',
 'РДБанк',
 'Дорожный банк',
 'Russian public joint-stock commercial roads Bank',
 'RosDorBank',
 'РосДорБанк',
 'roads Bank',
 'Акционерная компания "АЛРОСА" (публичное акционерное общество)',
 'ALRS',
 'ALRS RX',
 'алроса',
 'alrosa',
 'Акционерный Коммерческий банк "АВАНГАРД" - публичное акционерное общество',
 'AVAN',
 'Авангард',
 'AVANGARD',
 'акционерный коммерческий банк "Приморье"  (публичное акционерное общество)',
 'PRMB',
 'АКБ Приморье',
 'АКБ "Приморье"',
 'банк "Приморье"',
 'банк Приморье',
 'primbank',
 'PJSCB "Primorye"',
 'PJSCB Primorye'

In [73]:
company_names_lower = [company_name.lower() for company_name in company_names]
company_names_lower

['акционерный коммерческий банк "держава" публичное акционерное общество',
 'derzp',
 'держава',
 'derzhava',
 'derz',
 '"московский кредитный банк" (публичное акционерное общество)',
 'cbom',
 'cbom rx',
 'московский кредитный банк',
 'мкб',
 'credit bank of moscow',
 'credit bank',
 '"российский акционерный коммерческий дорожный банк" (публичное акционерное общество)',
 'rdrb',
 'российский акционерный коммерческий дорожный банк',
 'рдбанк',
 'дорожный банк',
 'russian public joint-stock commercial roads bank',
 'rosdorbank',
 'росдорбанк',
 'roads bank',
 'акционерная компания "алроса" (публичное акционерное общество)',
 'alrs',
 'alrs rx',
 'алроса',
 'alrosa',
 'акционерный коммерческий банк "авангард" - публичное акционерное общество',
 'avan',
 'авангард',
 'avangard',
 'акционерный коммерческий банк "приморье"  (публичное акционерное общество)',
 'prmb',
 'акб приморье',
 'акб "приморье"',
 'банк "приморье"',
 'банк приморье',
 'primbank',
 'pjscb "primorye"',
 'pjscb primorye'

In [74]:
dfsetiments

MessageID   ChannelID  issuerid  SentimentScore           DateAdded  \
0        241407  1203560567       153               2 2023-05-12 19:03:20   
1         33684  1136626166       230               4 2023-02-03 20:56:29   
2         10090  1063908560       118               4 2023-06-02 19:18:37   
3         10090  1063908560       220               5 2023-06-02 19:18:37   
4          9826  1063908560        89               2 2023-04-24 17:51:38   
...         ...         ...       ...             ...                 ...   
9284      47482  1197210433       157               4 2023-03-20 14:53:14   
9285     233829  1203560567       157               4 2023-03-20 14:58:04   
9286       9789  1063908560       225               3 2023-04-19 17:51:56   
9287     233867  1203560567       127               3 2023-03-20 14:58:04   
9288      14315  1075101206       112               5 2023-02-07 14:56:26   

              DatePosted                                        MessageText  \
0    2023-05-12 19:02:42  ⚠️🇷🇺#SELG #дивиденд  сд Селигдар: дивиденды 20...   
1    2023-02-03 16:46:34  Ozon продолжает развивать специализированные ф...   
2    2023-06-02 18:50:00  ​Фокусы продолжаются🔥Акции и инвестиции  📈ВТБ ...   
3    2023-06-02 18:50:00  ​Фокусы продолжаются🔥Акции и инвестиции  📈ВТБ ...   
4    2023-04-24 13:54:00  ​​Windfall Tax — налог на сверхприбыль. Какие ...   
...                  ...                                                ...   
9284 2023-03-20 12:15:21  #FLOT #Дивиденды 💰 7% — возможная дивдоходност...   
9285 2023-03-20 12:05:49  🇷🇺#FLOT #отчетность  ЧИСТАЯ ПРИБЫЛЬ СОВКОМФЛОТ...   
9286 2023-04-19 15:32:00  ​​Ключевой принцип создания портфеля  🔹Диверси...   
9287 2023-03-20 14:33:32  "💥🇷🇺#PLZL #листинг #торги  ""Полюс"" ведет диа...   
9288 2023-02-07 14:28:09  Роснефть (ROSN) справедливая цена 425.17 рубля...   

      IsForward  
0         False  
1         False  
2         False  
3         False  
4         False  
...         ...  
9284      False  
9285      False  
9286      False  
9287      False  
9288      False  

[9289 rows x 8 columns]

In [75]:
messages_and_issuerid = dfsetiments.groupby('MessageText')['issuerid'].apply(list).reset_index()
messages_and_issuerid

MessageText issuerid
0                                                          [235]
1     " США вводят санкции против российских банков:...    [100]
2     """АКРОН"" БУДЕТ ОСПАРИВАТЬ РЕШЕНИЕ ПОЛЬШИ ПО ...     [24]
3     """АЛРОСА"" может выплатить дивиденды по итога...      [4]
4     """Алекперов один был готов заплатить за долю ...    [236]
...                                                 ...      ...
7183  🪨 Взгляд на компанию: «Мечел» — эффект от отме...     [99]
7184  🪨 Взгляд на компанию: «Мечел»: 3-й кв. 2023 г....     [99]
7185  🪨 Мечел: акции с потенциалом роста свыше 90% д...     [99]
7186  🪨 Мечел: анализ ключевых тем. Взгляд БКС  Мы п...     [99]
7187  🫶 АФК Система обещает дивы. Верим?  Основатель...     [26]

[7188 rows x 2 columns]

In [76]:
messages_and_SentimentScore = dfsetiments.groupby('MessageText')['SentimentScore'].apply(list).reset_index()
messages_and_SentimentScore

MessageText SentimentScore
0                                                                  [4]
1     " США вводят санкции против российских банков:...            [2]
2     """АКРОН"" БУДЕТ ОСПАРИВАТЬ РЕШЕНИЕ ПОЛЬШИ ПО ...            [2]
3     """АЛРОСА"" может выплатить дивиденды по итога...            [4]
4     """Алекперов один был готов заплатить за долю ...            [4]
...                                                 ...            ...
7183  🪨 Взгляд на компанию: «Мечел» — эффект от отме...            [4]
7184  🪨 Взгляд на компанию: «Мечел»: 3-й кв. 2023 г....            [5]
7185  🪨 Мечел: акции с потенциалом роста свыше 90% д...            [4]
7186  🪨 Мечел: анализ ключевых тем. Взгляд БКС  Мы п...            [4]
7187  🫶 АФК Система обещает дивы. Верим?  Основатель...            [4]

[7188 rows x 2 columns]

In [77]:
messages_sentiments_company = messages_and_issuerid.copy()
messages_sentiments_company['SentimentScores'] = dfsetiments.groupby('MessageText')['SentimentScore'].apply(list).reset_index()['SentimentScore']
messages_sentiments_company

MessageText issuerid  \
0                                                          [235]   
1     " США вводят санкции против российских банков:...    [100]   
2     """АКРОН"" БУДЕТ ОСПАРИВАТЬ РЕШЕНИЕ ПОЛЬШИ ПО ...     [24]   
3     """АЛРОСА"" может выплатить дивиденды по итога...      [4]   
4     """Алекперов один был готов заплатить за долю ...    [236]   
...                                                 ...      ...   
7183  🪨 Взгляд на компанию: «Мечел» — эффект от отме...     [99]   
7184  🪨 Взгляд на компанию: «Мечел»: 3-й кв. 2023 г....     [99]   
7185  🪨 Мечел: акции с потенциалом роста свыше 90% д...     [99]   
7186  🪨 Мечел: анализ ключевых тем. Взгляд БКС  Мы п...     [99]   
7187  🫶 АФК Система обещает дивы. Верим?  Основатель...     [26]   

     SentimentScores  
0                [4]  
1                [2]  
2                [2]  
3                [4]  
4                [4]  
...              ...  
7183             [4]  
7184             [5]  
7185             [4]  
7186             [4]  
7187             [4]  

[7188 rows x 3 columns]

In [78]:
messages_sentiments_company['n_companies_in_message'] = messages_sentiments_company['issuerid'].apply(list).reset_index()['issuerid'].apply(len)

In [79]:
messages_sentiments_company

MessageText issuerid  \
0                                                          [235]   
1     " США вводят санкции против российских банков:...    [100]   
2     """АКРОН"" БУДЕТ ОСПАРИВАТЬ РЕШЕНИЕ ПОЛЬШИ ПО ...     [24]   
3     """АЛРОСА"" может выплатить дивиденды по итога...      [4]   
4     """Алекперов один был готов заплатить за долю ...    [236]   
...                                                 ...      ...   
7183  🪨 Взгляд на компанию: «Мечел» — эффект от отме...     [99]   
7184  🪨 Взгляд на компанию: «Мечел»: 3-й кв. 2023 г....     [99]   
7185  🪨 Мечел: акции с потенциалом роста свыше 90% д...     [99]   
7186  🪨 Мечел: анализ ключевых тем. Взгляд БКС  Мы п...     [99]   
7187  🫶 АФК Система обещает дивы. Верим?  Основатель...     [26]   

     SentimentScores  n_companies_in_message  
0                [4]                       1  
1                [2]                       1  
2                [2]                       1  
3                [4]                       1  
4                [4]                       1  
...              ...                     ...  
7183             [4]                       1  
7184             [5]                       1  
7185             [4]                       1  
7186             [4]                       1  
7187             [4]                       1  

[7188 rows x 4 columns]

In [80]:
messages_sentiments_company.sort_values(by='n_companies_in_message')

MessageText  \
0                                                         
4662  🇷🇺#ABIO  После интервью Марии Воронцовой о раз...   
4661  🇷🇺#ABIO  «Артген биотех» отчитался за девять м...   
4660  🇷🇺#5G #связь #телекомы #россия  МТС приступила...   
4658  🇷🇺 Российский рынок акций в среду снова прибав...   
...                                                 ...   
5753  💡 Подборка лучших акций российских компаний дл...   
2559  Какие интриги еще остались в дивидендном сезон...   
522   "✍️ Про рынок: Какие акции выбирают телеграм-к...   
3323  Топ-30 российских компаний по капитализации. П...   
4581  ❗️🇷🇺🇺🇸 #рынки #ожидания #россия #сша  Нефтегаз...   

                                               issuerid  \
0                                                 [235]   
4662                                              [266]   
4661                                              [266]   
4660                                              [100]   
4658                                              [225]   
...                                                 ...   
5753  [115, 127, 127, 112, 47, 160, 150, 36, 152, 11...   
2559  [235, 228, 150, 225, 89, 36, 90, 163, 103, 99,...   
522   [111, 223, 7, 89, 236, 112, 237, 254, 116, 160...   
3323  [111, 230, 11, 47, 160, 163, 4, 152, 126, 185,...   
4581  [111, 53, 33, 223, 241, 72, 136, 42, 187, 226,...   

                                        SentimentScores  \
0                                                   [4]   
4662                                                [4]   
4661                                                [3]   
4660                                                [4]   
4658                                                [4]   
...                                                 ...   
5753            [4, 5, 4, 4, 4, 4, 5, 4, 4, 4, 2, 4, 4]   
2559            [3, 3, 3, 3, 2, 3, 2, 4, 3, 2, 4, 4, 3]   
522    [4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 5, 4, 4, 4, 4]   
3323  [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...   
4581  [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...   

      n_companies_in_message  
0                          1  
4662                       1  
4661                       1  
4660                       1  
4658                       1  
...                      ...  
5753                      13  
2559                      13  
522                       16  
3323                      17  
4581                      77  

[7188 rows x 4 columns]

In [81]:
print(messages_sentiments_company.sort_values(by='n_companies_in_message').iloc[-1]['SentimentScores'])
messages_sentiments_company.sort_values(by='n_companies_in_message').iloc[-1]['MessageText']

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


'❗️🇷🇺🇺🇸 #рынки #ожидания #россия #сша  Нефтегаз🛢 Газпром (GAZP) Роснефть (ROSN) ГазпромНефть (SIBN) Новатэк (NVTK) Сургутнефтегаз (SNGSP) Лукойл (LKOH) Татнефть (TATN) Башнефть (BANEP) Транснефть (TRNFP) Саратовский НПЗ (KRKNP)  Финансы💵 Сбербанк (SBER) Банк Санкт-Петербург (BSPB) ВТБ (VTBR) Тинькофф (TCSG) Мосбиржа (MOEX) СПБ Биржа (SPBE) QIWI (QIWI) Ренессанс страхование (RENI) АФК СИСТЕМА (AFKS) SFI (SFIN)  ИТ🌐 Yandex (YNDX) Positive Technologies (POSI) HeadHunter (HHR) VK/MAIL (VKCO) Softline (SFTL)  Удобрения и нефтехимия 🧪 Акрон (AKRN) ФосАгро (PHOR) КуйбышевАзот (KAZT) Нижнекамскнефтехим (NKNCP) Казаньоргсинтез (KZOS) Нижнекамскшина (NKSH)  Ритейл 🛒 OZON (OZON) Магнит (MGNT) Fix price (FIXP) Beluga Group(BELU) Абрау Дюрсо (ABRD)  Детский Мир (DSKY)  Лента (LNTA) М.Видео-Эльдорадо (MVID) Окей (OKEY) X5 Group (FIVE) Обувь России (ORUP)  Энергетика ⚡️ ИнтерРАО (IRAO) Россети Центр (MRKC) Россети Центра и Приволжья (MRKP) Юнипро (UPRO) ОГК-2 (OGKB) ТГК-1 (TGKA)  Русгидро (HYDR) Энел

In [82]:
messages_sentiments_company.sort_values(by='n_companies_in_message').iloc[1]

MessageText               🇷🇺#ABIO  После интервью Марии Воронцовой о раз...
issuerid                                                              [266]
SentimentScores                                                         [4]
n_companies_in_message                                                    1
Name: 4662, dtype: object

In [83]:
messages_sentiments_company.sort_values(by='n_companies_in_message').iloc[1]['issuerid']

[266]

In [84]:
text = messages_sentiments_company.sort_values(by='n_companies_in_message').iloc[1]['MessageText']
entities_id = messages_sentiments_company.sort_values(by='n_companies_in_message').iloc[1]['issuerid']

clean_text = re.sub(r'[^\w\s]', '', text)

tokenizer = nlp.tokenizer

text = clean_text.lower()
tokens = tokenizer(text)

print(text)

for entity_id in entities_id:
    for entity_name in dict_company_names[entity_id]:
        # if entity_name != 'Артген Биотех':
        #     continue
        window_words = len(tokenizer(entity_name))
        print(entity_name, window_words)
        count_sybols = 0
        new_line = []
        for i in range(len(tokens) - window_words + 1):
            token_texts = [token.text for token in tokens[i:i+window_words]]
            str_entity_name = ''.join(entity_name).lower()
            str_token_texts = ' '.join(token_texts)
            levenshtein_ratio = Levenshtein.ratio(str_entity_name, str_token_texts)
            count_sybols1 = count_sybols if count_sybols==0 else count_sybols-1
            if levenshtein_ratio>0.9: 
                print(levenshtein_ratio) 
                print(count_sybols1,(count_sybols + len(str_token_texts)))
                print(tokens.text[count_sybols1:(count_sybols + len(str_token_texts))])
                print()
            
            count_sybols += len(tokens[i].text)+1

abio  после интервью марии воронцовой о развитии биотехнологий в россии в секторе генетики началась охота за активами рфарм протек и фармаимпекс интересуются покупкой лидера отрасли компании артген биотех  brief
Artgen biotech 2
ABIO 1
1.0
0 4
abio

Артген Биотех 2
1.0
191 205
артген биотех 

Артген 1
1.0
191 198
артген 

iАРТГЕН 1
0.9230769230769231
191 198
артген 



In [85]:
messages_sentiments_company.sort_values(by='n_companies_in_message')

MessageText  \
0                                                         
4662  🇷🇺#ABIO  После интервью Марии Воронцовой о раз...   
4661  🇷🇺#ABIO  «Артген биотех» отчитался за девять м...   
4660  🇷🇺#5G #связь #телекомы #россия  МТС приступила...   
4658  🇷🇺 Российский рынок акций в среду снова прибав...   
...                                                 ...   
5753  💡 Подборка лучших акций российских компаний дл...   
2559  Какие интриги еще остались в дивидендном сезон...   
522   "✍️ Про рынок: Какие акции выбирают телеграм-к...   
3323  Топ-30 российских компаний по капитализации. П...   
4581  ❗️🇷🇺🇺🇸 #рынки #ожидания #россия #сша  Нефтегаз...   

                                               issuerid  \
0                                                 [235]   
4662                                              [266]   
4661                                              [266]   
4660                                              [100]   
4658                                              [225]   
...                                                 ...   
5753  [115, 127, 127, 112, 47, 160, 150, 36, 152, 11...   
2559  [235, 228, 150, 225, 89, 36, 90, 163, 103, 99,...   
522   [111, 223, 7, 89, 236, 112, 237, 254, 116, 160...   
3323  [111, 230, 11, 47, 160, 163, 4, 152, 126, 185,...   
4581  [111, 53, 33, 223, 241, 72, 136, 42, 187, 226,...   

                                        SentimentScores  \
0                                                   [4]   
4662                                                [4]   
4661                                                [3]   
4660                                                [4]   
4658                                                [4]   
...                                                 ...   
5753            [4, 5, 4, 4, 4, 4, 5, 4, 4, 4, 2, 4, 4]   
2559            [3, 3, 3, 3, 2, 3, 2, 4, 3, 2, 4, 4, 3]   
522    [4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 5, 4, 4, 4, 4]   
3323  [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...   
4581  [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...   

      n_companies_in_message  
0                          1  
4662                       1  
4661                       1  
4660                       1  
4658                       1  
...                      ...  
5753                      13  
2559                      13  
522                       16  
3323                      17  
4581                      77  

[7188 rows x 4 columns]

In [86]:
messages_sentiments_company

MessageText issuerid  \
0                                                          [235]   
1     " США вводят санкции против российских банков:...    [100]   
2     """АКРОН"" БУДЕТ ОСПАРИВАТЬ РЕШЕНИЕ ПОЛЬШИ ПО ...     [24]   
3     """АЛРОСА"" может выплатить дивиденды по итога...      [4]   
4     """Алекперов один был готов заплатить за долю ...    [236]   
...                                                 ...      ...   
7183  🪨 Взгляд на компанию: «Мечел» — эффект от отме...     [99]   
7184  🪨 Взгляд на компанию: «Мечел»: 3-й кв. 2023 г....     [99]   
7185  🪨 Мечел: акции с потенциалом роста свыше 90% д...     [99]   
7186  🪨 Мечел: анализ ключевых тем. Взгляд БКС  Мы п...     [99]   
7187  🫶 АФК Система обещает дивы. Верим?  Основатель...     [26]   

     SentimentScores  n_companies_in_message  
0                [4]                       1  
1                [2]                       1  
2                [2]                       1  
3                [4]                       1  
4                [4]                       1  
...              ...                     ...  
7183             [4]                       1  
7184             [5]                       1  
7185             [4]                       1  
7186             [4]                       1  
7187             [4]                       1  

[7188 rows x 4 columns]

In [87]:
"Алекперов один был готов заплатить за долю в Яндексе без рассрочек и"[(44, 52)]

<>:1: SyntaxWarning: str indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:1: SyntaxWarning: str indices must be integers or slices, not tuple; perhaps you missed a comma?
C:\Users\fluke\AppData\Local\Temp\ipykernel_15292\2902957727.py:1: SyntaxWarning: str indices must be integers or slices, not tuple; perhaps you missed a comma?
  "Алекперов один был готов заплатить за долю в Яндексе без рассрочек и"[(44, 52)]


TypeError: string indices must be integers, not 'tuple'

In [ ]:
dfsetiments.head(20)

MessageID   ChannelID  issuerid  SentimentScore           DateAdded  \
0      241407  1203560567       153               2 2023-05-12 19:03:20   
1       33684  1136626166       230               4 2023-02-03 20:56:29   
2       10090  1063908560       118               4 2023-06-02 19:18:37   
3       10090  1063908560       220               5 2023-06-02 19:18:37   
4        9826  1063908560        89               2 2023-04-24 17:51:38   
5        9826  1063908560       127               2 2023-04-24 17:51:38   
6        9826  1063908560       150               2 2023-04-24 17:51:38   
7        9826  1063908560         7               2 2023-04-24 17:51:38   
8        9672  1063908560       160               2 2023-04-04 14:44:28   
9        9694  1063908560       254               4 2023-04-06 20:44:23   
10       9694  1063908560       227               3 2023-04-06 20:44:23   
11       9694  1063908560       230               4 2023-04-06 20:44:23   
12       9694  1063908560       116               2 2023-04-06 20:44:23   
13       2974  1160971567       115               3 2023-04-01 14:43:35   
14       2974  1160971567       204               3 2023-04-01 14:43:35   
15       2974  1160971567       220               3 2023-04-01 14:43:35   
16       2974  1160971567       136               4 2023-04-01 14:43:35   
17       2974  1160971567       218               3 2023-04-01 14:43:35   
18       2974  1160971567       251               4 2023-04-01 14:43:35   
19       2974  1160971567        56               3 2023-04-01 14:43:35   

            DatePosted                                        MessageText  \
0  2023-05-12 19:02:42  ⚠️🇷🇺#SELG #дивиденд  сд Селигдар: дивиденды 20...   
1  2023-02-03 16:46:34  Ozon продолжает развивать специализированные ф...   
2  2023-06-02 18:50:00  ​Фокусы продолжаются🔥Акции и инвестиции  📈ВТБ ...   
3  2023-06-02 18:50:00  ​Фокусы продолжаются🔥Акции и инвестиции  📈ВТБ ...   
4  2023-04-24 13:54:00  ​​Windfall Tax — налог на сверхприбыль. Какие ...   
5  2023-04-24 13:54:00  ​​Windfall Tax — налог на сверхприбыль. Какие ...   
6  2023-04-24 13:54:00  ​​Windfall Tax — налог на сверхприбыль. Какие ...   
7  2023-04-24 13:54:00  ​​Windfall Tax — налог на сверхприбыль. Какие ...   
8  2023-04-04 11:35:09  У нас было 2 пакета префов Сургутнефтегаза, 75...   
9  2023-04-06 18:54:08  "​​""Бесплатных"" долларов больше нет🔥Акции и ...   
10 2023-04-06 18:54:08  "​​""Бесплатных"" долларов больше нет🔥Акции и ...   
11 2023-04-06 18:54:08  "​​""Бесплатных"" долларов больше нет🔥Акции и ...   
12 2023-04-06 18:54:08  "​​""Бесплатных"" долларов больше нет🔥Акции и ...   
13 2023-04-01 12:00:01  Итоги недели 27.03.23 - 01.04.23  Закончился п...   
14 2023-04-01 12:00:01  Итоги недели 27.03.23 - 01.04.23  Закончился п...   
15 2023-04-01 12:00:01  Итоги недели 27.03.23 - 01.04.23  Закончился п...   
16 2023-04-01 12:00:01  Итоги недели 27.03.23 - 01.04.23  Закончился п...   
17 2023-04-01 12:00:01  Итоги недели 27.03.23 - 01.04.23  Закончился п...   
18 2023-04-01 12:00:01  Итоги недели 27.03.23 - 01.04.23  Закончился п...   
19 2023-04-01 12:00:01  Итоги недели 27.03.23 - 01.04.23  Закончился п...   

    IsForward  
0       False  
1       False  
2       False  
3       False  
4       False  
5       False  
6       False  
7       False  
8       False  
9       False  
10      False  
11      False  
12      False  
13      False  
14      False  
15      False  
16      False  
17      False  
18      False  
19      False

In [ ]:
def fix_intervals(tuples):
    sorted_tuples = sorted(tuples)

    # Инициализируем список результатов
    result = []

    # Проходим по отсортированным кортежам
    for interval in sorted_tuples:
        # Если список результатов пуст или текущий интервал не пересекается с последним интервалом в списке результатов
        if not result or interval[0] > result[-1][1]:
            result.append(interval)
        else:
            # Иначе обновляем последний интервал в списке результатов
            result[-1] = (result[-1][0], max(result[-1][1], interval[1]))
    
    return result

In [ ]:
" США вводят санкции против российских банков  Московский кредитный банк Банк Урасиб USBN МТС Банк MTSS Банк Са"[90:94]

'ТС Б'

In [90]:
messages_sentiments_company

MessageText issuerid  \
0                                                          [235]   
1     " США вводят санкции против российских банков:...    [100]   
2     """АКРОН"" БУДЕТ ОСПАРИВАТЬ РЕШЕНИЕ ПОЛЬШИ ПО ...     [24]   
3     """АЛРОСА"" может выплатить дивиденды по итога...      [4]   
4     """Алекперов один был готов заплатить за долю ...    [236]   
...                                                 ...      ...   
7183  🪨 Взгляд на компанию: «Мечел» — эффект от отме...     [99]   
7184  🪨 Взгляд на компанию: «Мечел»: 3-й кв. 2023 г....     [99]   
7185  🪨 Мечел: акции с потенциалом роста свыше 90% д...     [99]   
7186  🪨 Мечел: анализ ключевых тем. Взгляд БКС  Мы п...     [99]   
7187  🫶 АФК Система обещает дивы. Верим?  Основатель...     [26]   

     SentimentScores  n_companies_in_message  
0                [4]                       1  
1                [2]                       1  
2                [2]                       1  
3                [4]                       1  
4                [4]                       1  
...              ...                     ...  
7183             [4]                       1  
7184             [5]                       1  
7185             [4]                       1  
7186             [4]                       1  
7187             [4]                       1  

[7188 rows x 4 columns]

In [103]:
text = "Газпром нефть ушел из ozon"
entities_id = [47, 230]
sentiment_scores = [1,2]

if text:

    clean_text = re.sub(r'[^\w\s]', '', text)


    text = clean_text.strip()
    tokens = tokenizer(text)

    # print(text)
    for entity_index, entity_id in enumerate(entities_id):
        indexes_array = []
        if entity_id in dict_company_names:
            for entity_name in dict_company_names[entity_id]:
                window_words = len(tokenizer(entity_name))
                count_sybols = 0
                new_line = []
                str_entity_name = ''.join(entity_name).lower()
                
                for i in range(len(tokens) - window_words + 1):
                    token_texts = [token.text for token in tokens[i:i+window_words]]
                    str_token_texts = ' '.join(token_texts).lower()
                    levenshtein_ratio = Levenshtein.ratio(str_entity_name, str_token_texts)
                    count_sybols1 = count_sybols if count_sybols==0 else count_sybols-1
                    if levenshtein_ratio>0.8: 
                        indexes_array.append((count_sybols1, (count_sybols + len(str_token_texts))))
                    
                    count_sybols += len(tokens[i].text)+1
        indexes_array_fix = fix_intervals(indexes_array)

        # print(entities_id)
        # print(indexes_array_fix)
        parts_of_text = []
        for i in range(len(indexes_array_fix)):
            left = indexes_array_fix[i][0]
            last_right = indexes_array_fix[i-1][1] if i>=1 else 0 
            parts_of_text.append(text[last_right:left])
            if i == len(indexes_array_fix)-1:
                right = indexes_array_fix[i][1]
                parts_of_text.append(text[right:len(text)])

        print([sentiment_scores[entity_index]," company_name ".join(parts_of_text)])


[1, ' company_name  ушел из ozon']
[2, 'Газпром нефть ушел из company_name ']


In [124]:
tokenizer = nlp.tokenizer
training_data1 = []
n=0

df = pd.DataFrame(columns=['issuerid', 'sentiment_score', 'message_text'])

for row in tqdm.tqdm(messages_sentiments_company.iterrows()):
    text = row[1]['MessageText']
    entities_id = row[1]['issuerid']
    entities_id = row[1]['issuerid']
    sentiment_scores = row[1]['SentimentScores']

    if text:

        clean_text = re.sub(r'[^\w\s]', '', text)


        text = clean_text.strip()
        tokens = tokenizer(text)

        # print(text)
        for entity_index, entity_id in enumerate(entities_id):
            indexes_array = []
            if entity_id in dict_company_names:
                for entity_name in dict_company_names[entity_id]:
                    window_words = len(tokenizer(entity_name))
                    count_sybols = 0
                    new_line = []
                    str_entity_name = ''.join(entity_name).lower()
                    
                    for i in range(len(tokens) - window_words + 1):
                        token_texts = [token.text for token in tokens[i:i+window_words]]
                        str_token_texts = ' '.join(token_texts).lower()
                        levenshtein_ratio = Levenshtein.ratio(str_entity_name, str_token_texts)
                        count_sybols1 = count_sybols if count_sybols==0 else count_sybols-1
                        if levenshtein_ratio>0.8: 
                            indexes_array.append((count_sybols1, (count_sybols + len(str_token_texts))))
                        
                        count_sybols += len(tokens[i].text)+1
            indexes_array_fix = fix_intervals(indexes_array)

            # print(entities_id)
            # print(indexes_array_fix)
            parts_of_text = []
            for i in range(len(indexes_array_fix)):
                left = indexes_array_fix[i][0]
                last_right = indexes_array_fix[i-1][1] if i>=1 else 0 
                parts_of_text.append(text[last_right:left])
                if i == len(indexes_array_fix)-1:
                    right = indexes_array_fix[i][1]
                    parts_of_text.append(text[right:len(text)])
            if parts_of_text:
                df.loc[len(df)] = [entity_id, sentiment_scores[entity_index], " company_name ".join(parts_of_text)]

        n+=1
        if n==-1:
            break


7188it [01:16, 94.49it/s] 


In [125]:
df

issuerid  sentiment_score  \
0          100                2   
1           24                2   
2            4                4   
3          236                4   
4           36                4   
...        ...              ...   
8786        99                4   
8787        99                5   
8788        99                4   
8789        99                4   
8790        26                4   

                                           message_text  
0     США вводят санкции против российских банков  М...  
1      company_name  БУДЕТ ОСПАРИВАТЬ РЕШЕНИЕ ПОЛЬШИ...  
2      company_name  может выплатить дивиденды по ит...  
3     Алекперов один был готов заплатить за долю в c...  
4      company_name  групп в первом полугодии отгруз...  
...                                                 ...  
8786  Взгляд на компанию company_name   эффект от от...  
8787  Взгляд на компанию company_name  3й кв 2023 г ...  
8788   company_name  акции с потенциалом роста свыше...  
8789   company_name  анализ ключевых тем Взгляд БКС ...  
8790   company_name  обещает дивы Верим  Основатель ...  

[8791 rows x 3 columns]

In [126]:
df.to_csv("hakaton-gagarin-sentiment_interface\data\\preprocessed_data.csv")